In [1]:
import os
import numpy as np
import tensorflow as tf
from keras import models, layers

from loop import TrainingLoop
from batch_selection import windowed_batch_selector, sorting_batch_selector

In [2]:
# Loading the dataset from the files saved in the preprocessing notebook.
path = 'data/wine'
prefix = 'wine_'
X_train = np.load(os.path.join(path, prefix+'train_vectors.npy'))
y_train = np.load(os.path.join(path, prefix+'train_labels.npy'))
X_test  = np.load(os.path.join(path, prefix+'test_vectors.npy'))
y_test  = np.load(os.path.join(path, prefix+'test_labels.npy'))

In [3]:
def build_model():
    # Setting up the model.
    model = models.Sequential()
    model.add(layers.Dense(15, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))
    return model

In [4]:
def train(model, X_train, y_train, batch_selector, epochs):
    selector_str = {windowed_batch_selector: 'windowed', sorting_batch_selector: 'sorting', None: 'original'}
    print('\n\n'+selector_str[batch_selector]+'\n')
    # Put the model in our custom training loop.
    TrainingLoop(
        model, 
        X_train, 
        y_train, 
        validation_split = 0.1,
        batch_size = 8,
        optimizer = tf.keras.optimizers.Adam(),
        loss_function = tf.keras.losses.CategoricalCrossentropy(),
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        batch_selection = batch_selector,
        log_file = os.path.join('logs',selector_str[batch_selector],'wine_quality.csv')
    ).train(epochs)  # Training the model.

In [5]:
def evaluate(model, X_test, y_test):
    # We still have to compile the model for the test evaluation.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # After compiling we can run the evaluation.
    model.evaluate(X_test, y_test)

In [6]:
# Train and evaluate models with different batch selection algorithms.
for batch_selector in [None, windowed_batch_selector, sorting_batch_selector]:
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model = build_model()
    train(model, X_train, y_train, batch_selector=batch_selector, epochs=20)
    evaluate(model, X_test, y_test)



original

Epoch 1/20	Loss: 1.2288	Metrics: 0.3083: 	Validation metrics: 0.5: 	100% | 15/15 [00:03<00:00,  4.71it/s]
Epoch 2/20	Loss: 1.0411	Metrics: 0.3333: 	Validation metrics: 0.5: 	100% | 15/15 [00:00<00:00, 66.67it/s]
Epoch 3/20	Loss: 0.9005	Metrics: 0.4000: 	Validation metrics: 0.625: 	100% | 15/15 [00:00<00:00, 38.36it/s]
Epoch 4/20	Loss: 0.7932	Metrics: 0.5416: 	Validation metrics: 0.625: 	100% | 15/15 [00:00<00:00, 39.27it/s]
Epoch 5/20	Loss: 0.6862	Metrics: 0.6000: 	Validation metrics: 0.75: 	100% | 15/15 [00:00<00:00, 49.84it/s]
Epoch 6/20	Loss: 0.5969	Metrics: 0.6333: 	Validation metrics: 0.75: 	100% | 15/15 [00:00<00:00, 42.14it/s]
Epoch 7/20	Loss: 0.5225	Metrics: 0.6499: 	Validation metrics: 0.875: 	100% | 15/15 [00:00<00:00, 45.59it/s]
Epoch 8/20	Loss: 0.4646	Metrics: 0.6750: 	Validation metrics: 0.875: 	100% | 15/15 [00:00<00:00, 52.82it/s]
Epoch 9/20	Loss: 0.4231	Metrics: 0.7166: 	Validation metrics: 1.0: 	100% | 15/15 [00:00<00:00, 56.61it/s]
Epoch 10/20	Loss: 0.3914